In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
import nibabel as nib

def write_plyRGB(filename, vertices, faces, colorsR,colorsG,colorsB,comment=None):
    import pandas as pd
    print("writing ply format")
    # infer number of vertices and faces
    number_vertices = vertices.shape[0]
    number_faces = faces.shape[0]
    # make header dataframe
    header = ['ply',
            'format ascii 1.0',
            'comment %s' % comment,
            'element vertex %i' % number_vertices,
            'property float x',
            'property float y',
            'property float z',
            'property uchar red',
            'property uchar green',
            'property uchar blue',
            'element face %i' % number_faces,
            'property list uchar int vertex_indices',
            'end_header'
             ]
    header_df = pd.DataFrame(header)
    # make dataframe from vertices
    vertex_df = pd.DataFrame(vertices/50)
    #colors_df = pd.DataFrame(np.tile(np.round(colors/7*255), (3,1)).T)
    ColorsR_df=pd.DataFrame(colorsR)
    ColorsG_df=pd.DataFrame(colorsG)
    ColorsB_df=pd.DataFrame(colorsB)
    colorsConcat = pd.concat([ColorsR_df,ColorsG_df,ColorsB_df], axis=1)
    colors_df=pd.DataFrame(colorsConcat)
    colors_df=colorsConcat.astype(int)
    df_concat = pd.concat([vertex_df, colors_df], axis=1)
    # make dataframe from faces, adding first row of 3s (indicating triangles)
    triangles = np.reshape(3 * (np.ones(number_faces)), (number_faces, 1))
    triangles = triangles.astype(int)
    faces = faces.astype(int)
    faces_df = pd.DataFrame(np.concatenate((triangles, faces), axis=1))
    # write dfs to csv
    header_df.to_csv(filename, header=None, index=False)
    with open(filename, 'a') as f:
        df_concat.to_csv(f, header=False, index=False,
                         float_format='%.3f', sep=' ')
    with open(filename, 'a') as f:
        faces_df.to_csv(f, header=False, index=False,
                        float_format='%.0f', sep=' ')

In [ ]:
from neuromaps import datasets
from neuromaps.datasets import available_annotations
from neuromaps.datasets import fetch_annotation
from neuromaps.datasets import fetch_atlas

#for annotation in available_annotations(): # To view available annotations in neuromaps
    #print(annotation)


fslr = datasets.fetch_atlas(atlas='fslr', density='32k')
print(fslr.keys())


ltemp, rtemp = fslr['midthickness']
lvert, ltri = nib.load(ltemp).agg_data()
    
annotation = fetch_annotation(source='hcps1200',hemi='L',den='32k',desc='myelinmap',return_single=True)
myelin=nib.load(annotation).agg_data()



In [ ]:
cmap = matplotlib.cm.get_cmap('plasma')


colors=myelin
DataRange=np.array([np.percentile(myelin,25),np.percentile(myelin,75)]) # choose the upper and lower limits for the colorrange for the display
norm = matplotlib.colors.Normalize(vmin=DataRange[0], vmax=DataRange[1])


CMap=cmap(norm(colors))*255

write_plyRGB('Myelin.ply',lvert,ltri,CMap[:,0],CMap[:,1],CMap[:,2]) # This can be imported straight into Blender and has the same color layer as matplotlib



